This notebook demonstrates how to perfomr systematic the policy assessment and render the policy cards. 

In [69]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from res_ind_lib import *

#Imports the function to plot render the policy experiments
from policy_assessment import *

#ask jupyter to display the plots within this notebook
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data

## baseline data

In [70]:
#Load the excel data file into a dataframe (table)
df_original=pd.read_excel("inputs/all_data_compiled.xlsx", 
                 index_col=0, #tells Pandas to use the first column (Provinces) as the index of the table
                 skiprows=[0, 2] #The first (0) row is a description of the variables and the 3rd (2) row is empty. We skip them
                ).dropna() 
df_original.index.name="province"
#Displays the first rows of that dataframe
df_original=def_ref_values(df_original)


## policy experiment data

In [71]:
##POLICY DESCRIPTION
policy_info= pd.read_csv("inputs/policy_description.csv",index_col=0)

#Policy Labels (y axis)
policy_labels = policy_info.descriptor.copy() #we make a copy so we can modify it later
policy_labels.head()

#size of the policy
pol_increment = policy_info.increment    

# Parameters

In [72]:
colors = pd.read_csv("inputs/colors_pol_assess.csv", index_col=0)
colors

,color,edgecolor,alpha
dWtot_currency,#a6cee3,#1f78b4,1
dKtot,#cccccc,#969696,1


In [73]:
#unit for welfare and asset loses changes
unit = dict(multiplier = 1e-3, string = "million pesos per year"       )

In [74]:
pol_assess_set = colors.index

# Checking for out of bound problems

In [75]:
bounds= pd.read_csv("inputs/inputs_info.csv", index_col="key")
bounds.head(3)   

,descriptor,inf,sup
key,,,
avg_prod_k,Productivity of capital,0,10
dcap,Average consumption losses for poor people in ...,0,NaN
dcar,Average consumption losses for nonpoor people ...,0,NaN


In [76]:
clipped = check_bounds(df_original+pol_increment, bounds)

# Computing

In [77]:
deltas = compute_policies(df_original,pol_increment,pol_assess_set, bounds)
deltas.head()

Currently working on:  done.


province            Abra                Agusan del Norte                 \
outputs            dKtot dWtot_currency            dKtot dWtot_currency   
inputs                                                                    
T_rebuild_K     0.000000     109.260358         0.000000    2429.256339   
cp              0.050349      -0.115264         1.313773      -3.654982   
cr              0.122894      -0.086776         2.386512      -1.369782   
fap           934.765826    4282.111344      3090.609924   17219.581622   
far          5011.391772    7080.822550     19993.993601   22987.490978   

province    Agusan del Sur                         Aklan                 \
outputs              dKtot dWtot_currency          dKtot dWtot_currency   
inputs                                                                    
T_rebuild_K       0.000000    3246.123822       0.000000   93806.452212   
cp                2.048066      -5.669856      36.861058    -109.922946   
cr                2.395623      -1.594364     102.842465     -70.321943   
fap            4873.923349   27019.523749   22902.974842  137442.952301   
far           18469.531003   24606.277281  206022.211153  283524.076175   

province             Albay                     ...          Tawi-tawi  \
outputs              dKtot dWtot_currency      ...              dKtot   
inputs                                         ...                      
T_rebuild_K       0.000000   15330.277927      ...           0.000000   
cp                9.676592     -29.437917      ...           0.404334   
cr               11.982329      -7.409850      ...           1.314940   
fap           69321.589568  422158.326676      ...        1297.220325   
far          322287.036164  398905.939672      ...        8902.439089   

province                         Zambales                Zamboanga Sibugay  \
outputs     dWtot_currency          dKtot dWtot_currency             dKtot   
inputs                                                                       
T_rebuild_K    1077.966623       0.000000    8775.719011          0.000000   
cp               -0.706729       2.701583      -5.933268          5.750896   
cr               -1.047554      11.948419      -8.511142          5.890390   
fap            4535.696130   19380.762748   85190.023880      49398.224932   
far           14189.915911  217313.083627  309814.569804     172349.383763   

province                   Zamboanga del Norte                 \
outputs     dWtot_currency               dKtot dWtot_currency   
inputs                                                          
T_rebuild_K    9301.626681            0.000000   59702.109447   
cp              -18.163903           40.182721    -177.244321   
cr               -4.496491           27.798482     -21.476177   
fap          312359.485176        69637.896353  615404.104004   
far          263320.875886       192984.681260  298626.758117   

province    Zamboanga del Sur                 
outputs                 dKtot dWtot_currency  
inputs                                        
T_rebuild_K          0.000000    5814.488391  
cp                   2.883688      -8.513795  
cr                   5.382810      -3.658794  
fap               6532.981635   38661.236260  
far              40941.820428   55752.404128  

[5 rows x 166 columns]

# actual drawing

In [78]:
deltas.index

Index(['T_rebuild_K', 'cp', 'cr', 'fap', 'far', 'gdp_pc_pp', 'pi', 'pov_head',
       'protection', 'shewp', 'shewr', 'sigma_p', 'sigma_r', 'social_p',
       'social_r', 'v_p', 'v_r'],
      dtype='object', name='inputs')

In [79]:
render_pol_cards(
    deltas,
    colors,
    policy_labels,
    pol_increment,
    unit,
        [ "Maguindanao", "Quezon"])
#     df_original.index)


Currently working on:  Philippines


KeyError: 'Philippines'

# Merge all cards to single pdf

Merges the individual province cards into one big multi page pdf. Requires ghostscript.

In [ ]:
province_list=glob.glob("cards/*.pdf")
merge_cardfiles(province_list,"all_cards.pdf")

Converts the pdf files to png images. Requires Imagemagick.

In [ ]:
convert_pdf_to_png("cards/")

# Demo: policy experiments on the multi-hazard data

Here we demonstrate how to do policy experiments on the multi hazard data. This is just a demo as the data we have now prevents us to have meaningful experiments (river floods and costal floods)

In [ ]:
multi_hard_info=pd.read_csv("inputs/multi_hazard_data.csv", index_col=["province","hazard"])
multi_hard_info.head()

In [ ]:
fr=pd.read_csv("inputs/fa_ratios.csv", index_col=("province","hazard")).dropna()
fr.columns = fr.columns.astype(float)
fr.head()

## policy experiments on mh data

In [ ]:
##POLICY DESCRIPTION
policy_info_mh= pd.read_csv("inputs/policy_multihazard_description.csv",index_col=0)

#Policy Labels (y axis)
policy_labels_mh = policy_info_mh.descriptor.copy() #we make a copy so we can modify it later
policy_labels_mh.head()

#size of the policy
pol_increment_mh = policy_info_mh.increment    
pol_increment_mh.head()


## computation

In [ ]:
d_mh = compute_policies_mh(df_original,multi_hard_info,pol_increment_mh,pol_assess_set, bounds, fa_ratios=fr, multihazard_data =multi_hard_info)

## baseline mh data

In [ ]:
render_pol_cards(
    deltas.append(d_mh).drop("fap").drop("far"), #drop fap and far because superseded by multihazard info
    colors,
    policy_labels.append(policy_labels_mh),
    pol_increment.append(pol_increment_mh),
    unit,["Maguindanao"],
    outfolder="cards_coast_river/"
)

